In [1]:
from bertviz import head_view, model_view
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import AutoModelForSequenceClassification
### Required Packages
from termcolor import colored
import math
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from datetime import datetime
import pickle as pkl
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
import tqdm
import time
import transformers
from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline
use_cuda = torch.cuda.is_available()
import transformers


In [2]:
# model=BertForSequenceClassification.from_pretrained("./pytorch_model/", ignore_mismatched_sizes=True, output_attentions=True)
# torch.save(model, 'RNNmodel.pt')
# torch.save(model.state_dict(), 'RNNmodel.st')

In [2]:
# model = BertModel.from_pretrained("./test/pytorch_model/", output_attentions=True)
# model = BertModel.from_pretrained("./fineTuneModels/test/", output_attentions=True)
model = BertModel.from_pretrained("./fineTuneModels/fixed_first/", output_attentions=True)
# model = BertModel.from_pretrained("./fineTuneModels/fixed_last/", output_attentions=True)

Some weights of the model checkpoint at ./fineTuneModels/fixed_first/ were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# model.eval()

In [3]:
df = pd.read_csv('../processed_after.csv')

In [4]:
vocab = pkl.load(open("../medbert.types", 'rb'), encoding='bytes')
len(vocab.keys())

7226

In [5]:
# pt = df.loc[df["patient_id"]==523]
# pt = df.loc[df["patient_id"]==766]
# pt = df.loc[df["patient_id"]==927]
pt = df.loc[df["patient_id"]==14104]
# pt = df.loc[df["patient_id"]==15466]
pt = pt.loc[pt["Medical_code"].isin(vocab.keys())]
pt = pt.sort_values(by=['admission_date', 'diagnosis_priority'])
pt

,patient_id,admission_date,discharge_date,Medical_code,poa,diagnosis_priority,Billing_source
3659005,14104,2004-04-06,2004-04-06,ICD9_521.0,0,1,0
3659006,14104,2004-04-06,2004-04-06,ICD9_523.5,0,2,0
10828872,14104,2013-08-01,2013-08-01,ICD9_434.00,0,1,0
10828873,14104,2013-08-01,2013-08-01,ICD9_434.00,0,1,0
10365378,14104,2013-08-05,2013-08-05,ICD9_272.4,0,1,0
10365381,14104,2013-08-05,2013-08-05,ICD9_434.91,0,1,0
10365384,14104,2013-08-05,2013-08-05,ICD9_786.05,0,1,0
10365379,14104,2013-08-05,2013-08-05,ICD9_401.9,0,2,0
10365382,14104,2013-08-05,2013-08-05,ICD9_272.4,0,2,0
10365380,14104,2013-08-05,2013-08-05,ICD9_434.91,0,3,0


In [6]:
# tt = pt.head(25).reset_index()
# tt = tt[["patient_id", "Medical_code"]]
tt = set(pt["Medical_code"])
len(tt)

9

In [7]:
token_s = []
token_type_ids_s = []
token_s.append(0)
token_type_ids_s.append(0)
for k in tt:
    token_s.append(vocab[k])
    token_type_ids_s.append(0)
token_s.append(0)
token_type_ids_s.append(0)

token = []
token_type_ids = []
token.append(0)
token_type_ids.append(0)
for k in tt:
    token.append(vocab[k])
    token_type_ids.append(0)
token.append(0)
token_type_ids.append(0)
for k in tt:
    token.append(vocab[k])
    token_type_ids.append(1)
token.append(0)
token_type_ids.append(1)

In [10]:
# token_type_ids.index(1)

In [8]:
attention_s = model(torch.LongTensor(token_s).unsqueeze(0))[-1]

In [9]:
attention = model(torch.LongTensor(token).unsqueeze(0), token_type_ids=torch.LongTensor(token_type_ids).unsqueeze(0))[-1]

In [10]:
icd_en = {}
icd_zh = {}
icd = pd.DataFrame(columns=["ICD9", "en", "zh"])
with open('../ICD_original.csv', mode='r') as f:
    for i in f:
        if i == '\n':
            break
        i = i.replace('"', '')
        str = i.split(',')
        # str[0] = str[0].replace('"', '')
        if str[-1] != '':
            str[-1] = str[-1].replace('\n', '')
        # print("ICD9_"+str[0])

        icd_en["ICD9_"+str[0]] = ' '.join(str[1:-1])
        icd_zh["ICD9_"+str[0]] = str[-1]
        icd.loc[len(icd.index)] = ["ICD9_"+str[0], ' '.join(str[1:-1]), str[-1]]

In [11]:
CI_set = ["ICD9_997.02", "ICD9_434.00", "ICD9_434.01", "ICD9_434.10", "ICD9_434.11", "ICD9_434.90", "ICD9_434.91"]

In [12]:
CI = icd.loc[icd["ICD9"].isin(CI_set)]

In [13]:
ob_set = ["ICD9_540.0", "ICD9_540.1", "ICD9_540.9", "ICD9_541", "ICD9_542"]

In [31]:
ob = icd.loc[icd["ICD9"].isin(ob_set)]
ob

,ICD9,en,zh
4895,ICD9_540.0,Acute appendicitis with generalized peritonitis,急性闌尾炎併泛腹膜炎
4896,ICD9_540.1,Acute appendictis with peritoneal abscess,急性闌尾炎併腹膜膿瘍
4897,ICD9_540.9,Acute appendictis without mention of peritonitis,急性闌尾炎，未提及腹膜炎
4898,ICD9_541,Appendicitis unqualified,闌尾炎，性質不明者
4899,ICD9_542,Other appendicitis,其他闌尾炎


In [24]:
CI.assign(cases=[1724, 3915, 465, 565, 3301, 5818, 35])
CI

,ICD9,en,zh,cases
4279,ICD9_434.00,Cerebral thrombosis without mention of cerebra...,未提及腦梗塞之腦血栓症,1724
4280,ICD9_434.01,Cerebral thrombosis with cerebral infarction,腦血栓症合併腦梗塞,3915
4281,ICD9_434.10,Cerebral embolism without mention of cerebral ...,未提及腦梗塞之腦栓塞症,465
4282,ICD9_434.11,Cerebral embolism with cerebral infarction,伴有腦梗塞之腦栓塞症,565
4283,ICD9_434.90,Unspecified cerebral artery occlusion without ...,未提及腦梗塞之腦動脈阻塞,3301
4284,ICD9_434.91,Unspecified cerebral artery occlusion with cer...,伴有腦梗塞之腦動脈阻塞,5818
10843,ICD9_997.02,Iatrogenic cerebrovascular infarction or hemor...,腦血管梗塞或出血,35


In [14]:
tokens_s = []
tokens_s.append('[CLS]')
for k in tt:
    tokens_s.append(icd_zh[k])
tokens_s.append('[SEP]')

tokens = []
tokens.append('[CLS]')
for k in tt:
    tokens.append(icd_zh[k])
tokens.append('[SEP]')
for k in tt:
    tokens.append(icd_zh[k])
tokens.append('[SEP]')

In [15]:
head_view(attention_s, tokens_s)

<IPython.core.display.Javascript object>

In [80]:
head_view(attention, tokens, token_type_ids.index(1))

<IPython.core.display.Javascript object>

In [23]:
# model_view(attention, tokens)

<IPython.core.display.Javascript object>

In [37]:
model_view(attention, tokens, token_type_ids.index(1))

<IPython.core.display.Javascript object>

In [25]:
# model_type = 'bert'
# tokenizer = 
# show(model, model_type, tokenizer, sentence_a, sentence_b, layer=4, head=3)